# Discrete Choice Models

## Fair's Affair data

A survey of women only was conducted in 1974 by *Redbook* asking about extramarital affairs.

In [1]:
from __future__ import print_function
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols

In [2]:
print(sm.datasets.fair.SOURCE)


Fair, Ray. 1978. "A Theory of Extramarital Affairs," `Journal of Political
    Economy`, February, 45-61.

The data is available at http://fairmodel.econ.yale.edu/rayfair/pdf/2011b.htm



In [3]:
print( sm.datasets.fair.NOTE)


Number of observations: 6366
Number of variables: 9
Variable name definitions:

    rate_marriage   : How rate marriage, 1 = very poor, 2 = poor, 3 = fair,
                      4 = good, 5 = very good
    age             : Age
    yrs_married     : No. years married. Interval approximations. See
                      original paper for detailed explanation.
    children        : No. children
    religious       : How relgious, 1 = not, 2 = mildly, 3 = fairly,
                      4 = strongly
    educ            : Level of education, 9 = grade school, 12 = high school,
                      14 = some college, 16 = college graduate, 17 = some
                      graduate school, 20 = advanced degree
    occupation      : 1 = student, 2 = farming, agriculture; semi-skilled,
                      or unskilled worker; 3 = white-colloar; 4 = teacher
                      counselor social worker, nurse; artist, writers;
                      technician, skilled worker, 5 = managerial,
 

In [4]:
dta = sm.datasets.fair.load_pandas().data

In [5]:
dta['affair'] = (dta['affairs'] > 0).astype(float)
print(dta.head(10))

   rate_marriage  age  yrs_married  children  religious  educ  occupation  \
0              3   32          9.0       3.0          3    17           2   
1              3   27         13.0       3.0          1    14           3   
2              4   22          2.5       0.0          1    16           3   
3              4   37         16.5       4.0          3    16           5   
4              5   27          9.0       1.0          1    14           3   
5              4   27          9.0       0.0          2    14           3   
6              5   37         23.0       5.5          2    12           5   
7              5   37         23.0       5.5          2    12           2   
8              3   22          2.5       0.0          2    12           3   
9              3   27          6.0       0.0          1    16           3   

   occupation_husb   affairs  affair  
0                5  0.111111       1  
1                4  3.230769       1  
2                5  1.400000       

In [6]:
print(dta.describe())

       rate_marriage          age  yrs_married     children    religious  \
count    6366.000000  6366.000000  6366.000000  6366.000000  6366.000000   
mean        4.109645    29.082862     9.009425     1.396874     2.426170   
std         0.961430     6.847882     7.280120     1.433471     0.878369   
min         1.000000    17.500000     0.500000     0.000000     1.000000   
25%         4.000000    22.000000     2.500000     0.000000     2.000000   
50%         4.000000    27.000000     6.000000     1.000000     2.000000   
75%         5.000000    32.000000    16.500000     2.000000     3.000000   
max         5.000000    42.000000    23.000000     5.500000     4.000000   

              educ   occupation  occupation_husb      affairs       affair  
count  6366.000000  6366.000000      6366.000000  6366.000000  6366.000000  
mean     14.209865     3.424128         3.850141     0.705374     0.322495  
std       2.178003     0.942399         1.346435     2.203374     0.467468  
min    

In [7]:
affair_mod = logit("affair ~ occupation + educ + occupation_husb" 
                   "+ rate_marriage + age + yrs_married + children"
                   " + religious", dta).fit()

Optimization terminated successfully.
         Current function value: 0.545314
         Iterations 6


In [8]:
print(affair_mod.summary())

                           Logit Regression Results                           
Dep. Variable:                 affair   No. Observations:                 6366
Model:                          Logit   Df Residuals:                     6357
Method:                           MLE   Df Model:                            8
Date:                Sun, 01 May 2016   Pseudo R-squ.:                  0.1327
Time:                        12:14:43   Log-Likelihood:                -3471.5
converged:                       True   LL-Null:                       -4002.5
                                        LLR p-value:                5.807e-224
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           3.7257      0.299     12.470      0.000         3.140     4.311
occupation          0.1602      0.034      4.717      0.000         0.094     0.227
educ               -0.0392      

How well are we predicting?

In [9]:
affair_mod.pred_table()

array([[ 3882.,   431.],
       [ 1326.,   727.]])

The coefficients of the discrete choice model do not tell us much. What we're after is marginal effects.

In [10]:
mfx = affair_mod.get_margeff()
print(mfx.summary())

        Logit Marginal Effects       
Dep. Variable:                 affair
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
occupation          0.0293      0.006      4.744      0.000         0.017     0.041
educ               -0.0072      0.003     -2.538      0.011        -0.013    -0.002
occupation_husb     0.0023      0.004      0.541      0.589        -0.006     0.010
rate_marriage      -0.1308      0.005    -26.891      0.000        -0.140    -0.121
age                -0.0110      0.002     -5.937      0.000        -0.015    -0.007
yrs_married         0.0201      0.002     10.327      0.000         0.016     0.024
children           -0.0008      0.006     -0.134      0.893        -0.012     0.011
religious          -0.0685      0.006    -11.119      0.000        -0.081    -0.056


C:\Anaconda\lib\site-packages\statsmodels\discrete\discrete_model.py:430: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if exog == None:


In [11]:
respondent1000 = dta.ix[1000]
print(respondent1000)

rate_marriage       4.000000
age                37.000000
yrs_married        23.000000
children            3.000000
religious           3.000000
educ               12.000000
occupation          3.000000
occupation_husb     4.000000
affairs             0.521739
affair              1.000000
Name: 1000, dtype: float64


In [12]:
resp = dict(zip(range(1,9), respondent1000[["occupation", "educ", 
                                            "occupation_husb", "rate_marriage", 
                                            "age", "yrs_married", "children", 
                                            "religious"]].tolist()))
resp.update({0 : 1})
print(resp)

{0: 1, 1: 3.0, 2: 12.0, 3: 4.0, 4: 4.0, 5: 37.0, 6: 23.0, 7: 3.0, 8: 3.0}


In [13]:
mfx = affair_mod.get_margeff(atexog=resp)
print(mfx.summary())

        Logit Marginal Effects       
Dep. Variable:                 affair
Method:                          dydx
At:                           overall
                     dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
occupation          0.0400      0.008      4.711      0.000         0.023     0.057
educ               -0.0098      0.004     -2.537      0.011        -0.017    -0.002
occupation_husb     0.0031      0.006      0.541      0.589        -0.008     0.014
rate_marriage      -0.1788      0.008    -22.743      0.000        -0.194    -0.163
age                -0.0151      0.003     -5.928      0.000        -0.020    -0.010
yrs_married         0.0275      0.003     10.256      0.000         0.022     0.033
children           -0.0011      0.008     -0.134      0.893        -0.017     0.014
religious          -0.0937      0.009    -10.722      0.000        -0.111    -0.077


In [14]:
affair_mod.predict(respondent1000)

array([ 0.51878156])

In [15]:
affair_mod.fittedvalues[1000]

0.075161592850548464

In [16]:
affair_mod.model.cdf(affair_mod.fittedvalues[1000])

0.51878155721214347

The "correct" model here is likely the Tobit model. We have an work in progress branch "tobit-model" on github, if anyone is interested in censored regression models.

### Exercise: Logit vs Probit

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
support = np.linspace(-6, 6, 1000)
ax.plot(support, stats.logistic.cdf(support), 'r-', label='Logistic')
ax.plot(support, stats.norm.cdf(support), label='Probit')
ax.legend();

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
support = np.linspace(-6, 6, 1000)
ax.plot(support, stats.logistic.pdf(support), 'r-', label='Logistic')
ax.plot(support, stats.norm.pdf(support), label='Probit')
ax.legend();

Compare the estimates of the Logit Fair model above to a Probit model. Does the prediction table look better? Much difference in marginal effects?

### Genarlized Linear Model Example

In [ ]:
print(sm.datasets.star98.SOURCE)

In [ ]:
print(sm.datasets.star98.DESCRLONG)

In [ ]:
print(sm.datasets.star98.NOTE)

In [ ]:
dta = sm.datasets.star98.load_pandas().data
print(dta.columns)

In [ ]:
print(dta[['NABOVE', 'NBELOW', 'LOWINC', 'PERASIAN', 'PERBLACK', 'PERHISP', 'PERMINTE']].head(10))

In [ ]:
print(dta[['AVYRSEXP', 'AVSALK', 'PERSPENK', 'PTRATIO', 'PCTAF', 'PCTCHRT', 'PCTYRRND']].head(10))

In [ ]:
formula = 'NABOVE + NBELOW ~ LOWINC + PERASIAN + PERBLACK + PERHISP + PCTCHRT '
formula += '+ PCTYRRND + PERMINTE*AVYRSEXP*AVSALK + PERSPENK*PTRATIO*PCTAF'

#### Aside: Binomial distribution

Toss a six-sided die 5 times, what's the probability of exactly 2 fours?

In [ ]:
stats.binom(5, 1./6).pmf(2)

In [ ]:
from scipy.misc import comb
comb(5,2) * (1/6.)**2 * (5/6.)**3

In [ ]:
from statsmodels.formula.api import glm
glm_mod = glm(formula, dta, family=sm.families.Binomial()).fit()

In [ ]:
print(glm_mod.summary())

The number of trials 

In [ ]:
glm_mod.model.data.orig_endog.sum(1)

In [ ]:
glm_mod.fittedvalues * glm_mod.model.data.orig_endog.sum(1)

First differences: We hold all explanatory variables constant at their means and manipulate the percentage of low income households to assess its impact
on the response variables:

In [ ]:
exog = glm_mod.model.data.orig_exog # get the dataframe

In [ ]:
means25 = exog.mean()
print(means25)

In [ ]:
means25['LOWINC'] = exog['LOWINC'].quantile(.25)
print(means25)

In [ ]:
means75 = exog.mean()
means75['LOWINC'] = exog['LOWINC'].quantile(.75)
print(means75)

In [ ]:
resp25 = glm_mod.predict(means25)
resp75 = glm_mod.predict(means75)
diff = resp75 - resp25

The interquartile first difference for the percentage of low income households in a school district is:

In [ ]:
print("%2.4f%%" % (diff[0]*100))

In [ ]:
nobs = glm_mod.nobs
y = glm_mod.model.endog
yhat = glm_mod.mu

In [ ]:
from statsmodels.graphics.api import abline_plot
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, ylabel='Observed Values', xlabel='Fitted Values')
ax.scatter(yhat, y)
y_vs_yhat = sm.OLS(y, sm.add_constant(yhat, prepend=True)).fit()
fig = abline_plot(model_results=y_vs_yhat, ax=ax)

#### Plot fitted values vs Pearson residuals

Pearson residuals are defined to be 

$$\frac{(y - \mu)}{\sqrt{(var(\mu))}}$$

where var is typically determined by the family. E.g., binomial variance is $np(1 - p)$

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, title='Residual Dependence Plot', xlabel='Fitted Values',
                          ylabel='Pearson Residuals')
ax.scatter(yhat, stats.zscore(glm_mod.resid_pearson))
ax.axis('tight')
ax.plot([0.0, 1.0],[0.0, 0.0], 'k-');

#### Histogram of standardized deviance residuals with Kernel Density Estimate overlayed

The definition of the deviance residuals depends on the family. For the Binomial distribution this is 

$$r_{dev} = sign\left(Y-\mu\right)*\sqrt{2n(Y\log\frac{Y}{\mu}+(1-Y)\log\frac{(1-Y)}{(1-\mu)}}$$

They can be used to detect ill-fitting covariates

In [ ]:
resid = glm_mod.resid_deviance
resid_std = stats.zscore(resid) 
kde_resid = sm.nonparametric.KDEUnivariate(resid_std)
kde_resid.fit()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, title="Standardized Deviance Residuals")
ax.hist(resid_std, bins=25, normed=True);
ax.plot(kde_resid.support, kde_resid.density, 'r');

#### QQ-plot of deviance residuals

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
fig = sm.graphics.qqplot(resid, line='r', ax=ax)